In [ ]:
# Install required libraries
!pip install langchain
!pip install crewai
!pip install openai
!pip install python-dotenv

# Check if installations are successful
print("Libraries installed successfully!")

In [ ]:
import json

# Simulated lead and product data
data = {
    "lead": {
        "name": "John Doe",
        "job_title": "Marketing Director",
        "company": "Tech Solutions Inc.",
        "interests": ["AI tools", "marketing automation", "data analytics"]
    },
    "product": {
        "name": "AI-Powered Marketing Suite",
        "description": "A comprehensive tool to automate and optimize marketing campaigns using AI.",
        "key_benefits": ["Increases ROI by 30%", "Saves 10 hours per week", "Personalized campaign suggestions"]
    }
}

# Save to a JSON file
with open("lead_product_data.json", "w") as f:
    json.dump(data, f, indent=4)

# Verify file creation
print("JSON data file created successfully!")
!cat lead_product_data.json

In [ ]:
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables (for API key)
load_dotenv()

# For Colab, manually set your OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

# Initialize the LLM (OpenAI in this case)
llm = OpenAI(temperature=0.7)

# Test LLM connection
try:
    response = llm("Hello, are you working?")
    print("LLM Connection Successful:", response)
except Exception as e:
    print("Error in LLM Connection:", e)

In [ ]:
from crewai import Agent, Task, Crew
from langchain.tools import BaseTool

# Define a custom tool for email generation (optional, for advanced reasoning)
class EmailGenerationTools(BaseTool):
    name = "Email Content Analyzer"
    description = "Analyzes lead data and product details to assist in email content creation."

    def run(self, query: str) -> str:
        return "Analyzing lead data for personalized content..."

# Define the Agent for email generation
email_writer = Agent(
    role="Email Content Writer",
    goal="Craft personalized emails for leads based on their profile and product details.",
    backstory="You are an expert in writing compelling sales emails tailored to the recipient's interests and needs.",
    verbose=True,
    llm=llm
)

# Define tasks for the agent (subject and body generation)
task_subject = Task(
    description="""
    Analyze the lead's data (name, job title, company, interests) and product details.
    Generate a compelling email subject line that grabs attention.
    Return only the subject line.
    """,
    agent=email_writer,
    expected_output="A single compelling email subject line."
)

task_body = Task(
    description="""
    Use the lead's data (name: {name}, job title: {job_title}, company: {company}, interests: {interests})
    and product details (product name: {product_name}, description: {description}, benefits: {benefits}).
    Write a personalized email body that highlights how the product addresses the lead's needs or interests.
    Keep it concise, professional, and engaging. Return only the email body content.
    """,
    agent=email_writer,
    expected_output="A personalized email body."
)

# Placeholder for dynamic data injection (we will format this later)
print("CrewAI Agent and Tasks defined successfully!")

In [ ]:
# Modular structure for the application
class EmailGenerator:
    def __init__(self, data_path):
        self.data_path = data_path
        self.lead_data = None
        self.product_data = None

    def load_data(self):
        """Load data from JSON file."""
        with open(self.data_path, 'r') as f:
            data = json.load(f)
        self.lead_data = data["lead"]
        self.product_data = data["product"]
        print("Data loaded successfully!")
        return self.lead_data, self.product_data

    def generate_email(self, crew):
        """Generate email using CrewAI."""
        # Format task descriptions with actual data
        task_body.description = task_body.description.format(
            name=self.lead_data["name"],
            job_title=self.lead_data["job_title"],
            company=self.lead_data["company"],
            interests=", ".join(self.lead_data["interests"]),
            product_name=self.product_data["name"],
            description=self.product_data["description"],
            benefits=", ".join(self.product_data["key_benefits"])
        )

        # Execute tasks with CrewAI
        crew = Crew(
            agents=[email_writer],
            tasks=[task_subject, task_body],
            verbose=2
        )
        result = crew.kickoff()
        return result

# Instantiate the generator
email_gen = EmailGenerator("lead_product_data.json")
email_gen.load_data()
print("Modular design implemented successfully!")

In [ ]:
# Run the email generation
try:
    result = email_gen.generate_email(None)  # Crew is defined in the previous cell
    print("Generated Email Content:")
    print(result)
except Exception as e:
    print("Error in email generation:", e)

In [ ]:
# Write Dockerfile content (cannot run Docker in Colab, but we can create the file)
dockerfile_content = """
FROM python:3.9-slim

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install -r requirements.txt

# Copy application code
COPY . .

# Command to run the application
CMD ["python", "main.py"]
"""

# Write requirements.txt for Docker
requirements_content = """
langchain
crewai
openai
python-dotenv
"""

# Save files
with open("Dockerfile", "w") as f:
    f.write(dockerfile_content)
with open("requirements.txt", "w") as f:
    f.write(requirements_content)

# Verify file creation
print("Dockerfile and requirements.txt created successfully!")
!cat Dockerfile
!cat requirements.txt

In [ ]:
docker build -t email-generator .
docker run email

In [ ]:
# Write main.py content to run the application
main_content = """
import json
from crewai import Agent, Task, Crew
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables for API key
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set your OPENAI_API_KEY in the environment variables.")

# Initialize LLM
llm = OpenAI(temperature=0.7)

# EmailGenerator class for modularity
class EmailGenerator:
    def __init__(self, data_path):
        self.data_path = data_path
        self.lead_data = None
        self.product_data = None

    def load_data(self):
        with open(self.data_path, 'r') as f:
            data = json.load(f)
        self.lead_data = data['lead']
        self.product_data = data['product']
        return self.lead_data, self.product_data

    def generate_email(self, crew):
        task_body.description = task_body.description.format(
            name=self.lead_data['name'],
            job_title=self.lead_data['job_title'],
            company=self.lead_data['company'],
            interests=', '.join(self.lead_data['interests']),
            product_name=self.product_data['name'],
            description=self.product_data['description'],
            benefits=', '.join(self.product_data['key_benefits'])
        )
        result = crew.kickoff()
        return result

# Define Agent and Tasks
email_writer = Agent(
    role='Email Content Writer',
    goal='Craft personalized emails for leads based on their profile and product details.',
    backstory='You are an expert in writing compelling sales emails tailored to the recipient\\'s interests and needs.',
    verbose=True,
    llm=llm
)

task_subject = Task(
    description='Analyze the lead\\'s data (name, job title, company, interests) and product details. Generate a compelling email subject line that grabs attention. Return only the subject line.',
    agent=email_writer,
    expected_output='A single compelling email subject line.'
)

task_body = Task(
    description='Use the lead\\'s data (name: {name}, job title: {job_title}, company: {company}, interests: {interests}) and product details (product name: {product_name}, description: {description}, benefits: {benefits}). Write a personalized email body that highlights how the product addresses the lead\\'s needs or interests. Keep it concise, professional, and engaging. Return only the email body content.',
    agent=email_writer,
    expected_output='A personalized email body.'
)

# Setup Crew
crew = Crew(
    agents=[email_writer],
    tasks=[task_subject, task_body],
    verbose=2
)

# Run the application
if __name__ == '__main__':
    generator = EmailGenerator('lead_product_data.json')
    generator.load_data()
    print('Data loaded successfully!')
    result = generator.generate_email(crew)
    print('Generated Email Content:')
    print(result)
"""

# Save main.py
with open("main.py", "w") as f:
    f.write(main_content)

# Verify file creation
print("main.py created successfully!")
!cat main.py

In [ ]:
# Instructions for local Docker setup
print("""
Instructions for Running the Application with Docker Locally:

1. Ensure Docker is installed on your machine. Download it from https://www.docker.com/ if needed.
2. Download the following files to a local directory:
   - Dockerfile
   - requirements.txt
   - main.py
   - lead_product_data.json
3. Create a `.env` file in the same directory with your OpenAI API key:
   OPENAI_API_KEY=your-openai-api-key-here
4. Open a terminal in the directory containing these files.
5. Build the Docker image using the command:
   docker build -t email-generator .
6. Run the container with:
   docker run email-generator
7. The application will execute and output the generated email content.

Note: Ensure your OpenAI API key is valid and has sufficient quota for API calls.
""")

In [ ]:
# Final verification of requirements
requirements_check = {
    "Data Simulation": "✅ Satisfied (JSON file created with lead and product data in Step 2)",
    "LLM Integration": "✅ Satisfied (OpenAI API integration in Step 3, requires valid API key)",
    "Agentic Framework Usage": "✅ Satisfied (CrewAI used for email generation logic in Step 4)",
    "Modular Design": "✅ Satisfied (EmailGenerator class and clear structure in Step 5 and Step 8)",
    "Containerization with Docker": "✅ Satisfied (Dockerfile and instructions provided in Step 7 and Step 9)"
}

for req, status in requirements_check.items():
    print(f"{req}: {status}")